In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

In [8]:
from zipfile import ZipFile 

try:
    with ZipFile("dog-breed-identification.zip", 'r') as zippedFile: 
        zippedFile.extractall(path="dataset")
except Exception as e:
    print("No file Detected!")

No file Detected!
